In [1]:
import calitp
from calitp.tables import tbl
from siuba import *

import pandas as pd
import numpy as np
import geopandas as gpd

from ipyleaflet import Map, GeoJSON, projections, basemaps, GeoData, LayersControl, WidgetControl, GeoJSON
from ipywidgets import Text, HTML

import shapely
from shapely.geometry import LineString, MultiPoint
from shapely.ops import split, substring

import zlib
import datetime as dt

/Users/edasmalchi/anaconda3/envs/pd1/lib/python3.9/site-packages/google/auth/_default.py:68: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. We recommend you rerun `gcloud auth application-default login` and make sure a quota project is added. Or you can use service accounts instead. For more information about service accounts, see https://cloud.google.com/docs/authentication/
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


In [2]:
##definition of "peak" pending
am_peak = (6, 9)
pm_peak = (3, 7)

In [3]:
am_commute_hours = list(am_peak)
pm_commute_hours = list(pm_peak)
commute_hours = am_commute_hours + pm_commute_hours

In [4]:
def split_every_km(geometry):
    lines = []
    
    geometry = geometry.iloc[0]
    if hasattr(geometry, 'geoms'): ##check if MultiLineString
        linestrings = geometry.geoms
    else:
        linestrings = [geometry]
    for linestring in linestrings:
        for i in range(0, int(linestring.length), 1000):
    #         print(i)
            lines.append(substring(linestring, i, i+1000))
    return lines

In [5]:
def find_stop_with_most_routes(segment, stops, route_count_by_stop):
    stops_in_seg = gpd.clip(stops, segment.geometry) 
    max_routes_stop = (stops_in_seg
                   >> inner_join(_, route_count_by_stop, on = "stop_id")
                   >> filter(_.n_routes == _.n_routes.max())
                  ).iloc[[0]]
    segment['stop_id'] = max_routes_stop['stop_id'].iloc[0]
    segment['n_routes'] = max_routes_stop['n_routes'].iloc[0]
    return segment

In [6]:
def get_operator_views(itp_id):
        shapes = tbl.gtfs_schedule.shapes() >> filter(_.calitp_itp_id == int(itp_id)) >> collect()
        shapes = gpd.GeoDataFrame(shapes, 
                                  geometry = gpd.points_from_xy(shapes.shape_pt_lon, shapes.shape_pt_lat),
                                  crs = 'EPSG:4326').to_crs('EPSG:6414') ## https://epsg.io/6414 (meters)
        trips = tbl.gtfs_schedule.trips() >> filter(_.calitp_itp_id == int(itp_id)) >> collect()
        stop_times = tbl.gtfs_schedule.stop_times() >> filter(_.calitp_itp_id == int(itp_id)) >> collect()
        stops = (tbl.gtfs_schedule.stops() 
                 >> filter(_.calitp_itp_id == itp_id)
                 >> select(_.stop_id, _.stop_lat, _.stop_lon)
                 >> collect())
        
        return shapes, trips, stop_times, stops

In [7]:
def fix_arrival_time(gtfs_timestring):
    split = gtfs_timestring.split(':')
    hour = int(split[0])
    if hour >= 24:
        split[0] = str(hour - 24)
        corrected = (':').join(split)
        return corrected
    else:
        return gtfs_timestring

In [8]:
def single_shape_hqta(shapes, trips, stop_times, stops, route_count_by_stop, shape_id, hqta):
    
    single_shape = (shapes
         >> filter(_.shape_id == shape_id)
         >> mutate(shape_pt_sequence = _.shape_pt_sequence.astype('int64'))
         >> arrange(_.shape_pt_sequence) ##arrange, then convert to line to preserve order...
        )
    
    route_line = LineString(list(single_shape['geometry']))
    single_line = single_shape[['calitp_itp_id', 'shape_id', 'calitp_extracted_at']].iloc[[0]] ##preserve info cols
    single_line['geometry'] = route_line
    single_line = gpd.GeoDataFrame(single_line, crs='EPSG:6414')
        ## test a shapes df split into segments
    _debug = single_line
    if hqta.size != 0:
        already_calculated = hqta.dissolve(by='calitp_itp_id')
        single_line = single_line.overlay(already_calculated, how='difference') ##doesn't like this?
        if single_line.size == 0:
            print('no line')
            segments_with_max_stop = None
            return

    segmented = pd.DataFrame()
    for segment in split_every_km(single_line.geometry): ##working
        to_append = single_line.drop(columns=['geometry'])
        to_append['geometry'] = segment
        segmented = segmented.append(to_append)
        
    segmented = segmented.reset_index()
    segmented['segment_sequence'] = segmented.index.astype(str)
    
    segmented = segmented.astype({'calitp_itp_id': str})
    ## compute (hopefully unique) hash of segment id that can be used across routes/operators
    segmented['hqta_segment_id'] = segmented.apply(lambda x:
                        zlib.crc32((x.calitp_itp_id + x.shape_id + x.segment_sequence).encode('utf-8')),
                                                   axis=1)
    segmented.geometry = segmented.buffer(50) ##generous buffer for street/sidewalk width?
    
    try:
        segments_with_max_stop = segmented.apply(find_stop_with_most_routes, axis=1,
                                                 args = (stops, route_count_by_stop))
    except IndexError:
        segments_with_max_stop = None
        return ##no stops in segment

    max_stop_times = (stop_times 
                 >> select(_.stop_id, _.trip_id, _.departure_time)
                 >> inner_join(_, segments_with_max_stop, on = "stop_id")
                )
    max_stop_times['departure_time'] = max_stop_times['departure_time'].apply(fix_arrival_time)
    max_stop_times['departure_hour'] = max_stop_times['departure_time'].apply(lambda x:
                                                                    dt.datetime.strptime(x, '%H:%M:%S').hour)
    max_stop_times = max_stop_times >> filter(_.departure_hour.isin(commute_hours))
    max_stop_times['am_peak'] = max_stop_times['departure_hour'].apply(lambda x: x in am_commute_hours)
    max_stop_times['pm_peak'] = ~max_stop_times['am_peak']
    segment_peak_service = max_stop_times.groupby(['hqta_segment_id'])[['am_peak', 'pm_peak']].sum()
    ## convert to trips per hour
    segment_peak_service['am_peak'] = (segment_peak_service['am_peak'] / len(am_commute_hours)).round(1)
    segment_peak_service['pm_peak'] = (segment_peak_service['pm_peak'] / len(pm_commute_hours)).round(1)
    ## consider segment HQTA if stop within segment has at least 4 trips/hour in both peaks
    segment_peak_service['hq_transit_corr'] = segment_peak_service.apply(lambda x:
                              True if x.am_peak >= 4 and x.pm_peak >= 4 else False, axis=1)
    segment_peak_service = segment_peak_service.reset_index()
    single_hqta = segments_with_max_stop >> inner_join(_, segment_peak_service, on = 'hqta_segment_id')
    single_hqta.drop(columns=['calitp_extracted_at'], inplace=True)
    
    return single_hqta

In [9]:
def single_operator_hqta(views, hqta_df=None):  
    
    shapes, trips, stop_times, stops = views
    
    stops = gpd.GeoDataFrame(stops,
                             geometry = gpd.points_from_xy(stops.stop_lon, stops.stop_lat),
                             crs = 'EPSG:4326').to_crs('EPSG:6414') ## https://epsg.io/6414 (meters)
    distinct_routes = (trips
                   >> distinct(_.route_id, _.shape_id, _.direction_id, _keep_all = True)
                   >> select(_.calitp_itp_id, _.route_id, _.shape_id, _.direction_id, _.trip_id))
    route_count_by_stop = (stop_times
                         >> select(_.stop_id, _.trip_id)
                         >> inner_join(_, distinct_routes, on = "trip_id")
                         >> count(_.stop_id)
                         >> rename(n_routes = _.n)
                         >> arrange(-_.n_routes))

    hqta = gpd.GeoDataFrame()
    for shape_id in views[0]['shape_id'].unique():
        try:
            _result = single_shape_hqta(shapes, trips, stop_times, stops, route_count_by_stop, shape_id, hqta)
            hqta = hqta.append(_result)
        except:
            print(f'unable to calculate HQTA for shape_id {shape_id}')
        try:
            hqta = hqta.set_crs('EPSG:6414')
        except:
            continue
        
    return hqta

In [10]:
bbb_views = get_operator_views(300)

In [11]:
bbb_hqta = single_operator_hqta(bbb_views)

25387
...
25387


/Users/edasmalchi/anaconda3/envs/pd1/lib/python3.9/site-packages/pandas/core/frame.py:3607: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_item(key, value)


appending (7, 11)
25386
...
25386
no stops
25311
...
25311


/Users/edasmalchi/anaconda3/envs/pd1/lib/python3.9/site-packages/pandas/core/frame.py:3607: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_item(key, value)


appending (10, 11)
25313
...
25313


...

appending (8, 11)
25382
...
25382


/Users/edasmalchi/anaconda3/envs/pd1/lib/python3.9/site-packages/pandas/core/frame.py:3607: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_item(key, value)


appending (1, 11)
25384
...
25384


/Users/edasmalchi/anaconda3/envs/pd1/lib/python3.9/site-packages/pandas/core/frame.py:3607: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_item(key, value)


appending (8, 11)
25385
...
25385
no line
25398
...
25398
no stops
25399
...
25399
no stops
25400
...
25400
no stops
25402
...
25402
no stops
25403
...
25403
no line


### Mapping

In [12]:
def map_hqta(hqta):
    
    x = hqta['geometry'].to_crs('EPSG:4326').iloc[0].centroid.x
    y = hqta['geometry'].to_crs('EPSG:4326').iloc[0].centroid.y
    
    m = Map(basemap=basemaps.CartoDB.Positron, center=[y, x], zoom=11)
    geo_data_hq = GeoData(geo_dataframe = hqta[hqta['hq_transit_corr']].to_crs('EPSG:4326'),
                           style={'color': 'black', 'fillColor': '#3366cc',
                                        'opacity':0.3, 'weight':.5, 'dashArray':'2', 'fillOpacity':0.3},
                           hover_style={'fillColor': 'red' , 'fillOpacity': 0.2},
                           name = 'HQTA')

    geo_data_not_hq = GeoData(geo_dataframe = hqta[~hqta['hq_transit_corr']].to_crs('EPSG:4326'),
                           style={'color': 'black', 'fillColor': '#fec44f',
                                        'opacity':0.3, 'weight':.5, 'dashArray':'2', 'fillOpacity':0.3},
                           hover_style={'fillColor': 'red' , 'fillOpacity': 0.2},
                           name = 'non-HQTA')

    m.add_layer(geo_data_hq)
    m.add_layer(geo_data_not_hq)
    m.add_control(LayersControl())
    
    return m

## Initial Idea

* chop shapes into short segments, find the busiest stop in that segement and count trips there in am/pm peaks to determine if it's a high quality transit corridor

In [13]:
map_hqta(bbb_hqta)

Map(center=[34.02114835603139, -118.4793779516037], controls=(ZoomControl(options=['position', 'zoom_in_text',…

## Result

* Methodology approximates HQTAs for a single operator, but doesn't quite get there
    * Inaccurate at some intersections
* Still some code issues that would require more effort/expertise to track down
* But... probably better to try a more sophisticated approach, maybe [like this](https://github.com/mattwigway/sb827-analysis/)
    * I think I can get started adapting this to our data warehouse, and perhaps switching osmnx in for Valhalla
    * Will likely need coding help along the way
* Learned a lot about our data structures trying this out though!